In [1]:
# import necessary libraries for pre-processing
import numpy as np
import pandas as pd
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# import necessary libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# import necessary libraries for modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import torch
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# import necessary modules for model evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ar_poems = pd.read_csv("/kaggle/input/arabic-poetry/Arabic_poetry_dataset.csv")
ar_poems.head()

,id,category,poet_name,poem_title,poem_text
0,0,الإمارات,خلفان بن مصبح,بدت تختال في حُلل الجمالِ,بدت تختال في حُلل الجمالِ\nوجادت بالزيارة والو...
1,1,الإمارات,خلفان بن مصبح,يا طائر الشعر القرير,يا طائر الشعر القرير\nيا وحي إلهام الصدور\nأسع...
2,2,الإمارات,خلفان بن مصبح,بنت حجرات أرى من عجب,بنت حجرات أرى من عجب\nأن أرى فيك جمال العرب\nد...
3,3,الإمارات,خلفان بن مصبح,هذا الربيع بنور الحسن وافانا,هذا الربيع بنور الحسن وافانا\nوقد كسا الأرض با...
4,4,الإمارات,خلفان بن مصبح,روحي فداك وإن مُنحتُ صدوداً,روحي فداك وإن مُنحتُ صدوداً\nأخفاك ما بي أم أط...


In [3]:
print('dataset exploration:\n\n')
print(ar_poems.info())

dataset exploration:


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54944 entries, 0 to 54943
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54944 non-null  int64 
 1   category    54944 non-null  object
 2   poet_name   54944 non-null  object
 3   poem_title  54944 non-null  object
 4   poem_text   54944 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.1+ MB
None


In [4]:
# drop useless columns 
ar_poems = ar_poems.drop(columns=['id'])

In [5]:
print('dataset exploration: Final Form\n\n')
print(ar_poems.info())

dataset exploration: Final Form


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54944 entries, 0 to 54943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    54944 non-null  object
 1   poet_name   54944 non-null  object
 2   poem_title  54944 non-null  object
 3   poem_text   54944 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB
None


In [6]:
ar_poems.head()

,category,poet_name,poem_title,poem_text
0,الإمارات,خلفان بن مصبح,بدت تختال في حُلل الجمالِ,بدت تختال في حُلل الجمالِ\nوجادت بالزيارة والو...
1,الإمارات,خلفان بن مصبح,يا طائر الشعر القرير,يا طائر الشعر القرير\nيا وحي إلهام الصدور\nأسع...
2,الإمارات,خلفان بن مصبح,بنت حجرات أرى من عجب,بنت حجرات أرى من عجب\nأن أرى فيك جمال العرب\nد...
3,الإمارات,خلفان بن مصبح,هذا الربيع بنور الحسن وافانا,هذا الربيع بنور الحسن وافانا\nوقد كسا الأرض با...
4,الإمارات,خلفان بن مصبح,روحي فداك وإن مُنحتُ صدوداً,روحي فداك وإن مُنحتُ صدوداً\nأخفاك ما بي أم أط...


In [7]:
# explore unique categories in the dataset
ar_poems.category.unique()

array(['الإمارات', 'البحرين', 'الجزائر', 'السعودية', 'السودان', 'العراق',
       'المغرب', 'اليمن', 'تونس', 'سوريا', 'عمان', 'فلسطين', 'لبنان',
       'ليبيا', 'مصر', 'الأردن', 'الكويت', 'قطر', 'موريتانيا',
       'العصر الجاهلي', 'العصر الإسلامي', 'العصر العباسي',
       'العصر الايوبي', 'العصر العثماني', 'عصر المخضرمون', 'العصر الاموي',
       'العصر الأندلسي', 'العصر المملوكي'], dtype=object)

In [8]:
# remove the only row which misses "poem_text"
ar_poems.dropna(inplace=True)

In [9]:
modern = ['الإمارات', 'البحرين', 'الجزائر', 'السعودية', 'السودان', 'العراق',
       'المغرب', 'اليمن', 'تونس', 'سوريا', 'عمان', 'فلسطين', 'لبنان',
       'ليبيا', 'مصر', 'الأردن', 'الكويت', 'قطر', 'موريتانيا','شعراء العراق والشام',
       'أفغانستان', 'إيران']
modern

['الإمارات',
 'البحرين',
 'الجزائر',
 'السعودية',
 'السودان',
 'العراق',
 'المغرب',
 'اليمن',
 'تونس',
 'سوريا',
 'عمان',
 'فلسطين',
 'لبنان',
 'ليبيا',
 'مصر',
 'الأردن',
 'الكويت',
 'قطر',
 'موريتانيا',
 'شعراء العراق والشام',
 'أفغانستان',
 'إيران']

In [10]:
# merge the 22 different countries categories as 'العصر الحديث' category
for i in range(len(ar_poems)):
    if ar_poems.iloc[i]['category'] in modern:
        ar_poems.iloc[i]['category'] = 'العصر الحديث'

In [11]:
ar_poems.category.unique()

array(['العصر الحديث', 'العصر الجاهلي', 'العصر الإسلامي', 'العصر العباسي',
       'العصر الايوبي', 'العصر العثماني', 'عصر المخضرمون', 'العصر الاموي',
       'العصر الأندلسي', 'العصر المملوكي'], dtype=object)

In [12]:
ar_poems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54944 entries, 0 to 54943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    54944 non-null  object
 1   poet_name   54944 non-null  object
 2   poem_title  54944 non-null  object
 3   poem_text   54944 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [13]:
ar_poems.category.value_counts()

category
العصر الحديث      13409
العصر العباسي     10116
العصر المملوكي     9058
العصر الايوبي      6069
العصر العثماني     4622
العصر الاموي       4314
العصر الأندلسي     3238
عصر المخضرمون      2028
العصر الجاهلي      1878
العصر الإسلامي      212
Name: count, dtype: int64

In [14]:
ar_poems['poem_text'].loc[100]

'إذا كان صد منكموا قد بدا لنا\nبلا موجبٍ في القرب والبعد أجدر\nحفظنا وضيعتم ودمنا وزلتموا\nفكونوا كما شئتم فما نتغيى\nحرام علينا الصد عنكم وهجركم\nلأنا أناس في الهوى ليس نغدر'

In [15]:
# Load the dataset
df = ar_poems

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['poem_text'], df['category'], test_size=0.2, random_state=42)

# Encode target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Predict the categories of the test data
y_pred = model.predict(X_test_vec)

# Evaluate the performance of the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.22      0.33       598
           1       0.00      0.00      0.00        37
           2       0.52      0.45      0.48       846
           3       0.53      0.34      0.42      1237
           4       0.50      0.15      0.23       378
           5       0.56      0.82      0.67      2710
           6       0.43      0.59      0.50      1999
           7       0.68      0.25      0.36       977
           8       0.45      0.49      0.47      1808
           9       0.39      0.12      0.19       399

    accuracy                           0.51     10989
   macro avg       0.47      0.34      0.36     10989
weighted avg       0.52      0.51      0.48     10989



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

In [16]:
# calculate and print various evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f_score}")
print(f"Recall: {recall}")

Accuracy: 0.5063245063245063
F1 Score: 0.48070421230330057
Recall: 0.5063245063245063


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Assuming you have already loaded and preprocessed the data as shown in your code

# Function to predict the category of a poem
def predict_poem_category(poem_text):
    # Preprocess the input poem
    poem_vec = vectorizer.transform([poem_text])
    
    # Predict the category
    category_index = model.predict(poem_vec)[0]
    
    # Convert the category index back to its original label
    predicted_category = label_encoder.inverse_transform([category_index])[0]
    
    return predicted_category

# Example usage
input_poem = "يا طائر الشعر القرير"
predicted_category = predict_poem_category(input_poem)
print(f"The predicted category for the poem is: {predicted_category}")



The predicted category for the poem is: العصر الحديث


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder



# Function to predict the category of a poem
def predict_poem_category(poem_text):
    # Preprocess the input poem
    poem_vec = vectorizer.transform([poem_text])
    
    # Predict the category
    category_index = model.predict(poem_vec)[0]
    
    # Convert the category index back to its original label
    predicted_category = label_encoder.inverse_transform([category_index])[0]
    
    return predicted_category

# Example usage
input_poem = "ليكفك حاسدا حسده"
predicted_category = predict_poem_category(input_poem)
print(f"The predicted category for the poem is: {predicted_category}")



The predicted category for the poem is: العصر العباسي


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Assuming you have already loaded and preprocessed the data as shown in your code

# Function to predict the category of a poem
def predict_poem_category(poem_text):
    # Preprocess the input poem
    poem_vec = vectorizer.transform([poem_text])
    
    # Predict the category
    category_index = model.predict(poem_vec)[0]
    
    # Convert the category index back to its original label
    predicted_category = label_encoder.inverse_transform([category_index])[0]
    
    return predicted_category

# Example usage
input_poem = "لو كنت فينا ولها مغرما"
predicted_category = predict_poem_category(input_poem)
print(f"The predicted category for the poem is: {predicted_category}")



The predicted category for the poem is: العصر الحديث


In [20]:
import pickle

# Save the model to a file
with open('poem_category_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [21]:

# Load the model from a file
with open('poem_category_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
